# DellaVigna and Pope, 2018, "What Motivates Effort? Evidence and Expert Forecasts", Table 5, GMM

#### Authors:  

- Massimiliano Pozzi (Bocconi University, pozzi.massimiliano@studbocconi.it)
- Salvatore Nunnari (Bocconi University, salvatore.nunnari@unibocconi.it)

#### Description:

The code in this Jupyter notebook replicates columns 1 and 3 in Panel A of Table 5 and columns 1 and 4 in Panel B of Table 5. This estimates use minimum distance.

This notebook was tested with the following packages versions:
- Pozzi:   (Anaconda 4.10.3 on Windows 10 Pro) : python 3.8.3, numpy 1.18.5, pandas 1.0.5, sklearn 1.0
- Nunnari: (Anaconda 4.10.1 on macOS 10.15.7): python 3.8.10, numpy 1.20.2, pandas 1.2.4, scikit-learn 0.24.2

In [1]:
# Import the necessary libraries

import numpy as np
import pandas as pd
from sklearn.utils import resample

## 1. Data Cleaning and Data Preparation

We import the relevant dataset containing data on the number of buttonpresses in the different treatments and for different piece rates wage that the participants received when completing the task. We then compute the means for the treatments that are used to compute the minimum distance estimates.

In [36]:
# import the dataset

dt = pd.read_stata('../input/mturk_clean_data_short.dta')

# compute the rounded empirical moments in the different treatments

emp_moments = np.array(np.round(dt.groupby("treatment").mean("buttonpresses")))

We set up the bootstrap procedure for the standard errors. The cell below creates 'number' new samples and computes the mean for the relevant treatments. 

In [37]:
# Treatments are as follows:
# 1.1: benchmark specification with piece rate of 0.01 
# 1.2: benchmark specification with piece rate of 0.10 
# 1.3: benchmark specification with piece rate of 0.00 
# 3.1: social preferences (charity) with piece rate of 0.01 
# 3.2: social preferences (charity) with piece rate of 0.10 
# 10 : social preferences (gift exchange) bonus of 40 cents (independently of nr buttonpresses)
# 4.1: time discounting with extra 0.01 paid two weeks later
# 4.2: time discounting with extra 0.01 paid four weeks later

# resample is a useful command from the sklearn library that samples with replacement our data.
# We first get a smaller dataframe containing only observations for a specific treatment, we then resample the observations and
# compute the rounded mean of buttonpresses, save the result and then pass onto the next treatment. we do this for 'number' times.

def bootstrap(dataset, number):
    
    # define the vector containing the treatments (used for the loop) and the vectors that will store the mean buttonpresses in our 1000 new samples. 
    # E11 is a 1xnumber vector that will contain the average buttonpresses in each of our 'number' samples for treatment 1.1 etc.

    treatment = ['1.1', '1.2', '1.3', '3.1', '3.2', '10', '4.1', '4.2']
    E11, E12, E13, E31, E32, E10, E41, E42 = [], [], [], [], [], [], [], []

    for i in range(1, number+1):     # we want 'number' new samples
        for treat in treatment:      # we want to compute the mean for all relevant treatments
            db = dataset[dataset.treatment==treat] # keep only observations for treatment 'treat'
            bootsample = resample(db['buttonpresses'],replace=True,) # resample the dataset
            if treat == '1.1': E11.append(np.round(np.mean(bootsample)))
            if treat == '1.2': E12.append(np.round(np.mean(bootsample)))
            if treat == '1.3': E13.append(np.round(np.mean(bootsample)))
            if treat == '3.1': E31.append(np.round(np.mean(bootsample)))
            if treat == '3.2': E32.append(np.round(np.mean(bootsample)))
            if treat == '10' : E10.append(np.round(np.mean(bootsample)))
            if treat == '4.1': E41.append(np.round(np.mean(bootsample)))
            if treat == '4.2': E42.append(np.round(np.mean(bootsample)))
                
        if i == 500:  print('50% done')
        if i == 1000: print('100% done')
            
    return E11, E12, E13, E31, E32, E10, E41, E42

bt = bootstrap(dt,2000)
E11, E12, E13, E31, E32, E10, E41, E42 = bt[0], bt[1], bt[2], bt[3], bt[4], bt[5], bt[6], bt[7]

50% done
100% done


## 2. Model and Estimation Technique (Section 2 in the Paper)

The model is one of costly effort, where an agent needs to choose the optimal effort (in this case the number of buttons pressed in a 10 minute session) to solve a simple tradeoff problem between disutility of effort and consumption utility derived from the consequent payment. On top of this simple problem the authors use 18 different treatments to examine the effects of standard monetary incentives, behavioral factors like social preferences and reference dependence and non-monetary incentives. We briefly examine here the benchmark model and the solutions for the treatments used for the minimum distance estimates we compute.

The model for treatment 1.1, 1.2 and 1.3 can be written as follows:

$$ \max_{e\geq0} \;\; (s+p)e-c(e) $$

Where e is the number of buttons pressed, p is the piece-rate that varies across treatments, s a parameter that captures intrinsic motivation and c(e) is a convex cost function, either of power or exponential form:

$$ c(e)=\frac{ke^{1+\gamma}}{1+\gamma} \qquad \qquad c(e)=\frac{kexp(\gamma e)}{\gamma}$$

given this problem an optimal interior solution can be found trough the first order condition:

$$ e^*= c^{'-1}(s+p) \qquad \qquad e^*= \left(\frac{s+p}{k}\right)^{\frac{1}{\gamma}} \qquad \qquad e^*= \frac{1}{\gamma}\left(\frac{s+p}{k}\right) $$

where the first equation uses a generic cost function, the second one a power cost function and the last one an exponential cost function. The parameters to estimate are three: s, k and &gamma;. Given the fact that we have three different treatments with three different piece-rate we can estimate the parameters by imposing the theoretical moments found trought the first order condition to be equal to the empirical moments. This will lead to a system of three equations in three unknowns that is exactly identified. Once we have an estimate for s, k and &gamma; we use these to find the estimates for the parameters in the other treatments.

For the social preferences (charitable giving) treatments 3.1, 3.2 the optimal effort is:

$$ e^*= c^{'-1}(s+ \alpha p_{CH}+a*0.01) $$

where &alpha; is a parameter that multiplies the return to charity and a is a "warm glow" parameter, so that an individual cares about giving to charity but does not pay attention to the actual return. We have two different charity piece rate, so two different empirical moments that can be used to identify &alpha; and a.

For the social preferences (gift exchange) treatment 10 the optimal effort is:

$$ e^*= c^{'-1}(s+ \Delta s_{GE}) $$

where &Delta;s<sub>GE</sub> represents a possible increase of intrinsic motivation s because of the gift, reflecting positive reciprocity towards the employer.

Finally we have the time preferences treatments 4.1, 4.2 whose optimal effort is:

$$ e^*= c^{'-1}(s+ \beta \delta^t p) $$

where beta is the present bias parameter and delta is the standard time discounting. In this case we have two different t: two weeks and four weeks that allows us to identify the two parameters.

In [38]:
# Define the function to estimate the parameters using minimum distance. 
# There are two specifications: exponential cost function and power cost function
# The parameters are found by imposing the theoretical means found from the agent's problem being equal to the empirical mean found in the data.
# E11 up to E42 are the relevant empirical moments
# specification can either be 'Exp' or 'Power'
# P is a vector containing the different piece-rates 
# To simplify the computations the authors take the log of the variables to estimate gamma k and s.

def mindisest(E11,E12,E13,E31,E32,E10,E41,E42,specification):
    
    P=[0,0.01,0.1,]
    
    if specification == 'Exp':
        
        log_k = (np.log(P[2]) - np.log(P[1])*(E12)/(E11))/(1 - (E12)/(E11))
        log_gamma = np.log((np.log(P[1]) - log_k)/(E11))
        log_s = np.exp(log_gamma)*(E13) + log_k
        
    if specification == 'Power':
        
        log_k = (np.log(P[2]) - np.log(P[1])*np.log(E12)/np.log(E11))/(1 - np.log(E12)/np.log(E11))
        log_gamma = np.log((np.log(P[1]) - log_k)/np.log(E11))
        log_s = np.exp(log_gamma)*np.log(E13) + log_k
        
    k = np.exp(log_k)       # estimate for k
    g = np.exp(log_gamma)   # estimate for gamma
    s = np.exp(log_s)       # estimate for s
    
    if specification == 'Exp':
        
        EG31, EG32, EG10, EG41, EG42 = np.exp(E31*g), np.exp(E32*g), np.exp(E10*g), np.exp(E41*g), np.exp(E42*g)
        alpha = 100/9*k*(EG32-EG31)
        a = 100*k*EG31-100*s-alpha
        s_ge = k*EG10 - s
        delta = np.sqrt((k*EG42-s)/(k*EG41-s))
        beta  = 100*(k*EG41-s)/(delta**2)
        
    if specification == 'Power':
        
        alpha = 100/9*k*( E32**g-E31**g )
        a = 100*k*E31**g-100*s-alpha
        s_ge = k*E10**g - s
        delta = np.sqrt((k*E42**g-s)/(k*E41**g-s))
        beta  = 100*(k*E41**g-s)/(delta**2)
        
    return k, g, s, alpha, a, s_ge, beta, delta

# We vectorize the function so we can use as inputs the vectors containing the means in the different treatments to get the estimatesin our boostrap procedure

vmindisest = np.vectorize(mindisest)

## 3. Estimation

### Point Estimates and Standard Errors

We now compute the minimum distance estimates for Table 5 and the standard errors via a bootstrap procedure.

In [39]:
# Table 5 minimum distance estimates: columns (1) (3) panel A and columns (1) (4) panel B
    
Table5Exp = mindisest(emp_moments[0],emp_moments[1],emp_moments[2],emp_moments[6],emp_moments[7],emp_moments[4],emp_moments[8],emp_moments[9],'Exp')
Table5Exp = np.array(Table5Exp).flatten()
Table5Power = mindisest(emp_moments[0],emp_moments[1],emp_moments[2],emp_moments[6],emp_moments[7],emp_moments[4],emp_moments[8],emp_moments[9],'Power')
Table5Power = np.array(Table5Power).flatten()

In [40]:
# Store mean and standard error of estimates for the exponential cost function specification using the Bootstrap procedure

import warnings
warnings.filterwarnings('ignore') # This is to avoid showing RuntimeWarning in the notebook regarding overflow. For a couple of cases in our 1000 new samples
                                  # we cannot find the results because of overflow. Losing 2-3 observations out of thousands should not change the overall mean
                                  # for the parameters

estimatesExp = vmindisest(E11,E12,E13,E31,E32,E10,E41,E42,'Exp')
k_exp_mean, k_exp_sd = np.nanmean(estimatesExp[0]), np.nanstd(estimatesExp[0])
g_exp_mean, g_exp_sd = np.nanmean(estimatesExp[1]), np.nanstd(estimatesExp[1])
s_exp_mean, s_exp_sd = np.nanmean(estimatesExp[2]), np.nanstd(estimatesExp[2])
alpha_exp_mean, alpha_exp_sd = np.nanmean(estimatesExp[3]), np.nanstd(estimatesExp[3])
a_exp_mean, a_exp_sd = np.nanmean(estimatesExp[4]), np.nanstd(estimatesExp[4])
s_ge_exp_mean, s_ge_exp_sd = np.nanmean(estimatesExp[5]), np.nanstd(estimatesExp[5])
beta_exp_mean, beta_exp_sd = np.nanmean(estimatesExp[6]), np.nanstd(estimatesExp[6])
delta_exp_mean, delta_exp_sd = np.nanmean(estimatesExp[7]), np.nanstd(estimatesExp[7])

In [41]:
# Store mean and standard error of estimates for the power cost function specification using the Bootstrap procedure

estimatesPower = vmindisest(E11,E12,E13,E31,E32,E10,E41,E42,'Power')
k_power_mean, k_power_sd = np.nanmean(estimatesPower[0]), np.nanstd(estimatesPower[0])
g_power_mean, g_power_sd = np.nanmean(estimatesPower[1]), np.nanstd(estimatesPower[1])
s_power_mean, s_power_sd = np.nanmean(estimatesPower[2]), np.nanstd(estimatesPower[2])
alpha_power_mean, alpha_power_sd = np.nanmean(estimatesPower[3]), np.nanstd(estimatesPower[3])
a_power_mean, a_power_sd = np.nanmean(estimatesPower[4]), np.nanstd(estimatesPower[4])
s_ge_power_mean, s_ge_power_sd = np.nanmean(estimatesPower[5]), np.nanstd(estimatesPower[5])
beta_power_mean, beta_power_sd = np.nanmean(estimatesPower[6]), np.nanstd(estimatesPower[6])
delta_power_mean, delta_power_sd = np.nanmean(estimatesPower[7]), np.nanstd(estimatesPower[7])

In [42]:
# To obtain confidence intervals in panel B table 5. CI are the 2.5% and 97.5% quantiles of the distribution of our
# parameters vectors. Since there are 1000 values in each vector, the low/high end of the CI is in position 24 and 974 
# of our arrays. We obtain CI only for alpha, a, s_ge, beta, delta as in the paper

CI_Exp, CI_Power = [], []
for ci in range(3,8):
    a  = sorted(estimatesExp[ci])
    CI_Exp.append([a[24],a[974]])
for ci in range(3,8):
    a  = sorted(estimatesPower[ci])
    CI_Power.append([a[24],a[974]])

In [43]:
# Create two dataframes with our results. Table5Results contains point estimates and standard errors. CIpanelB contains 
# confidence intervals for the variables alpha, a, s_ge, beta and delta

params_name = ["Level k of cost of effort", "Curvature γ of cost function","Intrinsic motivation s","Social preferences α",
                "Warm glow coefficient a","Gift exchange Δs", "Present bias β","(Weekly) discount factor δ"]
               
sd_exp   = [k_exp_sd,g_exp_sd,s_exp_sd,alpha_exp_sd,a_exp_sd,s_ge_exp_sd,beta_exp_sd,delta_exp_sd]
sd_power = [k_power_sd,g_power_sd,s_power_sd,alpha_power_sd,a_power_sd,s_ge_power_sd,beta_power_sd,delta_power_sd]

Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort Power point estimates': Table5Power,
                              'Minimum dist est on average effort Power standard errors': sd_power,
                              'Minimum dist est on average effort Exp point estimates': Table5Exp,
                              'Minimum dist est on average effort Exp standard errors': sd_exp})
CIpanelB = pd.DataFrame({'CI_Exp':CI_Exp, 'CI_Power':CI_Power})

# Save the dataframe

Table5Results.to_csv('../output/table5GMM_python.csv', index=False)

In [44]:
# Print the results

# Formatting the results nicely for the table

from decimal import Decimal

columns = [Table5Power, sd_power, Table5Exp, sd_exp]
vs = []
for col in columns:
    col = ['{0:.2e}'.format(Decimal(col[0])), round(col[1],3), '{0:.2e}'.format(Decimal(col[2])),
           round(col[3],3), round(col[4],3), '{0:.2e}'.format(Decimal(col[5])), round(col[6],2), round(col[7],2)]
    vs.append(col)
    
Table5Results = pd.DataFrame({'Parameters name': params_name,
                              'Minimum dist est on average effort Power point estimates': vs[0],
                              'Minimum dist est on average effort Power standard errors': vs[1],
                              'Minimum dist est on average effort Exp point estimates': vs[2],
                              'Minimum dist est on average effort Exp standard errors': vs[3]})
    
# Standard errors are different since the seed we used for the bootstrap procedure is different from the one used by the authors since 
# random generation across softwares/languages is not easily replicated (each software uses its own algorithm)

from IPython.display import display
print('Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates')
display(Table5Results)

Table 5: Estimates of behavioural parameters I: Mturkers actual effort. Minimum distance estimates


,Parameters name,Minimum dist est on average effort Power point estimates,Minimum dist est on average effort Power standard errors,Minimum dist est on average effort Exp point estimates,Minimum dist est on average effort Exp standard errors
0,Level k of cost of effort,2.54e-112,3.68e-62,1.27e-16,2.33e-11
1,Curvature γ of cost function,33.138,12.065,0.016,0.006
2,Intrinsic motivation s,7.12e-7,1.03e-5,3.32e-6,2.43e-5
3,Social preferences α,0.003,0.014,0.003,0.014
4,Warm glow coefficient a,0.125,0.152,0.143,0.156
5,Gift exchange Δs,3.26e-6,2.49e-5,8.58e-6,3.92e-5
6,Present bias β,1.17,3.98,1.15,3.45
7,(Weekly) discount factor δ,0.75,0.3,0.76,0.29
